# Dependences

In [71]:
include("../../src/struct_data.jl")

save_data

# Model

In [72]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

dump(model)

  0.000000 seconds (4 allocations: 144 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 150000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_1"
    path_output: String ""


# Aggregate

In [73]:
agg = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time agg = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        ),
        AggType(
            "HEK_2", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    model
)
# show_aggregates(agg)

  0.816621 seconds (1.99 M allocations: 235.772 MiB, 3.50% gc time)


Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}), AggType("HEK_2", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2"]), Float32[-15.77 -4.62 -13.88; -13.77 -4.62 -13.88; … ; 14.77 4.62 13.88; 16.77 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.

# kNN

In [74]:
include("../../src/neighbor.jl")
println("------------------------ SIZE ------------------------")
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

println("------------------------ IDX -------------------------")
display(agg.Simulation.Neighbor.idx_red)

------------------------ SIZE ------------------------


1000×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 36  37  38  41  42  43  44  45  57  …  4965  4966  4967  4972  4973  4974
  6   7   8  49   1   2   3   3  58     5006  5007  5008  5002  5003  5004
  5   6   7  50  50   1   2  53  15     5007  5008  4959  5003  5004  5005
 42  43  44   5  51  51  52  54  10     4958  4959  4960  4965  4966  4967
 43  44  45  10   6  52  53   7  49     4957  4958  5004  4964  4965  4966
  2   3   2   9   4   7   8  14   4  …  5002  5003  4999  5007  5006  5007
 35   1  37  40  11   5   6  13  66     5004  5005  5000  4973  5008  4975
 37  36  39  42  10  12  13  44  50     4997  4998  4966  4971  4972  4973
 51  38  53  58  41  11  12  46  48     4998  4999  4968  4957  4974  4959
  0  52   0   0  43  42  43  62   0     4964  4965  4951     0  4958     0
  ⋮                   ⋮              ⋱                       ⋮        
  0   0   0   0   0   0   0   0   0        0     0     0     0     0     0
  0   0   0   0   0   0   0   0   0        0     0  

------------------------ IDX -------------------------


# Force

## Function

In [111]:
include("../../src/forces/forces.jl")

threads=(64,3)
@cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    shmem=prod(threads.+6)*sizeof(Float32),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(1),
        model.Time.dt
    )
)

display(agg.Simulation.Force.Pol')
display(agg.Position')
display(agg.Simulation.Force.F')
display(sum(agg.Simulation.Force.Pol .^ 2, dims=2)')

3×5008 adjoint(::CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}) with eltype Float32:
 -0.466321  -0.459895  -0.66219   …  0.731662   -0.752651  -0.119317
 -0.088526  -0.729209   0.968133     0.843948   -0.60649    0.181796
 -0.705329  -0.172056   0.236876     0.0908294  -0.657107  -0.912221

3×5008 adjoint(::CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}) with eltype Float32:
 -15.77  -13.77  -11.77  -18.77  -16.77  …  15.77  17.77  12.77  14.77  16.77
  -4.62   -4.62   -4.62   -2.89   -2.89      2.89   2.89   4.62   4.62   4.62
 -13.88  -13.88  -13.88  -13.88  -13.88     13.88  13.88  13.88  13.88  13.88

3×5008 adjoint(::CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}) with eltype Float32:
  7.13207f-12   7.13207f-12   7.13207f-12  …   7.13207f-12  -7.13207f-12
 -1.49224f-5   -1.49224f-5   -1.49224f-5       1.49224f-5    1.49224f-5
  0.000364654   0.000364654   0.000364654     -0.000364654  -0.000364654

1×5008 adjoint(::CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}) with eltype Float32:
 0.722781  0.772853  1.43189  1.38458  …  0.94887  1.25583  1.3661  0.879433